<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/helper_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# helper_lib

In [2]:
!pip install pypandoc==1.11

In [3]:
import pypandoc

def test_pypandoc():
  output = pypandoc.convert_text('# some title', 'rst', format='md')
  return output

#print(test_pypandoc())